In [2]:
import pandas as pd
import time
from datetime import datetime
from pandas import ExcelWriter 
pd.__version__




'1.0.0'

In [3]:
# temp=1588744463872/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

NameError: name 'temp' is not defined

In [ ]:
temp=1588744479826/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

In [4]:
    # Total number of seconds between dates

In [116]:
class JTLReader:
    def __init__(self,jtlPath):
        self.jtlPath=jtlPath
        self.jtlColumns = ['timeStamp', 'elapsed','label','threadName','dataType','bytes','sentBytes','URL','allThreads','success','responseCode','responseMessage']
        self.df_Jtl=pd.read_csv(self.jtlPath,usecols=self.jtlColumns,dtype={'label':'category'})#dtype={'label':'category'},nrows=195222    
        self.df_Jtl.sort_values(by=['timeStamp'],inplace=True) 
        self.df_Jtl.reset_index(drop=True, inplace=True)
        self.shape=self.df_Jtl.shape
        self.rows=self.shape[0]
        self.rows
        """Conver Epoc time to Actual Time Start"""
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Africa/Cairo')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')
        """Conver Epoc time to Actual Time End"""
        
        """Filtering Subscriber Txn"""
        #self.filt_Subscriber=(self.df_Jtl['label']=='iBORDETS_Subscribers')
        #self.df_Jtl=self.df_Jtl.loc[~self.filt_Subscriber]
        """Saving master data frame to export to excel"""
        
        
        """Filter Transaction Controller & HTTP Request"""
        self.filt=(pd.isnull(self.df_Jtl['URL'])) #& (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[self.filt]
        self.df_jtl_HTTPReqeust=self.df_Jtl.loc[~self.filt]
        """Error Handling To be updated"""
        self.error_filt=(self.df_jtl_HTTPReqeust['success']==False)
        self.df_HttpErrors=self.df_jtl_HTTPReqeust.loc[self.error_filt]
        
        
    
        
      
        
    """def getTransactionControllerData(self):
        print("Hello shiva")
        filt=(pd.isnull(self.df_Jtl['URL'])) & (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[filt]
        return self.df_jtl_TC"""
        

        
    def converTimeStamp(self):
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Africa/Cairo')#Africa/Cairo
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]') 
        
    def ms_to_sec(self):
         self.df_Jtl['elapsed']=((self.df_Jtl['elapsed'])/1000)
        
        
    def overAllResponseTimeSummary(self):
        self.df_overAllResponseTimeSummary=self.df_jtl_TC[['timeStamp','elapsed']]
        return self.df_overAllResponseTimeSummary
       
    def listOfErrors(self):
        return self.df_Jtl['responseCode'].value_counts()
    
    def shapeOfDaraFrame(self):
        return self.df_Jtl.shape
    
    def columnType(self,columnName):
        self.columnName=columnName
        return self.df_Jtl[self.columnName].dtype
    
    def getPeriod(self):
        self.startTime=self.df_Jtl['timeStamp'][0]
        self.endRow=self.df_Jtl['timeStamp'].shape[0]
        self.endTime=self.df_Jtl['timeStamp'][self.endRow-1]
        return f'{self.startTime} : {self.endTime}'
    
    def getDuration(self):
        self.startTime= self.df_Jtl['timeStamp'][0]
        self.endRow= self.df_Jtl['timeStamp'].shape[0]
        self.endTime= self.df_Jtl['timeStamp'][self.endRow-1]#-1
        self.executionStartTime=datetime(self.startTime.year,self.startTime.month,self.startTime.day,self.startTime.hour,self.startTime.minute,self.startTime.second)
        self.executionEndTime=datetime(self.endTime.year,self.endTime.month,self.endTime.day,self.endTime.hour,self.endTime.minute,self.endTime.second)
        self.timeDiff = self.executionEndTime-self.executionStartTime
        self.duration_in_s = self.timeDiff.total_seconds()
        self.duration=time.strftime("%H:%M:%S", time.gmtime(self.duration_in_s))
        return self.duration
    
    def getMaxRunningVusers(self):
        self.allThread=self.df_Jtl['allThreads'].max()
        return self.allThread 
    
      
    def getTotalHits(self):
        return self.df_jtl_HTTPReqeust.shape[0]
    
    def getAvgHitsPerSecond(self):
        return self.df_jtl_HTTPReqeust.shape[0]/self.duration_in_s
    
    def getTotalThroughput(self):
        self.totalThroughput=self.df_jtl_TC['bytes'].sum()
        return self.totalThroughput
    
    def getAvgThroughputPerSecond(self):
        self.avgThroughput=(self.df_jtl_TC['bytes'].sum())/self.duration_in_s
        return self.avgThroughput
    
    def getTotalPassFailFreqList(self):
        self.passFail_freq=self.df_jtl_TC['success'].value_counts().to_frame()
        #self.passFailFrequencyeqList=self.passFail_freq.index.tolist()
        
        return self.passFail_freq      
    
    

    def getAnalysisSummary(self):
        self.period=self.getPeriod()
        self.duration=self.getDuration()
        self.maxRunningVusers=self.getMaxRunningVusers()
        self.totalThroughput=self.getTotalThroughput()
        self.avgThroughputPerSecond=self.getAvgThroughputPerSecond()
        self.totalHits=self.getTotalHits()
        self.avgHitsPerSecond=self.getAvgHitsPerSecond()
        
        self.df_AnalysisSummary=pd.DataFrame(
            {
                'Data':[
                        'Period','Duration','Maximum Running Vusers','Total bytes',
                        'Average bytes/second','TotalHits','Average Hits Per Second',
                        'Total Error','Total Pass','Total Failed'
                ],
                'Values':[
                            self.period,self.duration,self.maxRunningVusers,self.totalThroughput,
                            self.avgThroughputPerSecond,self.totalHits,self.avgHitsPerSecond,
                            'NA',self.getTotalPassFailFreqList(),'NA'
                ]
                
            }
        )
        return self.df_AnalysisSummary
    
    def responseTimeSummary(self):
        self.df_responseTimeSummary=self.df_jtl_TC[['label','elapsed']]
        self.df_txnGroup=self.df_responseTimeSummary.groupby(['label'],observed=True )  #observed=True      
        self.df_txnGroup=self.df_txnGroup['elapsed'].describe(percentiles=[.90])#self.df_txnGroup.describe(percentiles=[.90])
        self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(0)#self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(1)
        self.df_txnGroup = self.df_txnGroup[['count','min','mean','90%','max',]]
        self.df_txnGroup.rename(columns={'count':'#Samples','min':'Min','mean':'Average','90%':'90 Percentile','max':'Max'},inplace=True)
        ##self.df_txnGroup=self.df_txnErrorGroup.reset_index() """To be enable if you run describe () on data frame. Now we are running on specific column"""
        ##self.df_txnGroup['throughput']=self.df_txnGroup['count']/self.duration_in_s
        ##self.validTxn_filt=(self.df_txnGroup['count']>0)
        self.df_responseTimeSummaryOutput=self.df_txnGroup #self.df_txnGroup[self.validTxn_filt]
       
        return self.df_responseTimeSummaryOutput
    
    def getPassFailSummary(self):
        self.df_passFailSummary=self.df_jtl_TC[['label','success']]
        self.df_passFailGroup=self.df_passFailSummary.groupby(['label'],observed=True)#whe we use observed=True-->refer https://stackoverflow.com/questions/54922327/valueerror-operands-could-not-be-broadcast-together-with-shape-when-calling-pan?answertab=votes#tab-top
        self.df_txnGroup=self.df_passFailGroup['success'].value_counts()
        self.df_txnGroupSlice=self.df_txnGroup.loc[:,]
        self.df_passFailSummaryOutput=self.df_txnGroupSlice.unstack()
        self.df_passFailSummaryOutput=self.df_passFailSummaryOutput.fillna(0)
        self.df_passFailSummaryOutput.rename(columns={True:'Pass',False:'Fail'},inplace=True)
        return self.df_passFailSummaryOutput
        
    def getFinalResponseSummary(self):
        rs=pd.merge(self.df_responseTimeSummaryOutput,self.df_passFailSummaryOutput,left_index=True,right_index=True)
        if 'Fail' in rs.columns:# If transaction fail observed 
            rs['Error%']=((rs['Fail'])/(rs['#Samples']))*100 
            rs=rs[['#Samples','Pass','Fail','Error%','Min','Average','90 Percentile','Max']]#'Fail','Error%'
            return rs
        else:# All transactions pass
            rs=rs[['#Samples','Pass','Min','Average','90 Percentile','Max']]
            
            return rs
            
            
    
    def getRunnerVuserGraphData(self):
        '''T=min,S=Seconds'''       
        self.df_vusers=self.df_jtl_TC[['timeStamp','allThreads']]
        self.df_vusers.sort_values(by=['timeStamp'])
        self.df_vusers.set_index('timeStamp',inplace=True)
        self.df_vusers_summary = pd.DataFrame()
        self.df_vusers_summary['Vusers']=self.df_vusers['allThreads'].resample('1T').max()  
        self.df_vusers_summary.ffill(inplace=True)
        return self.df_vusers_summary
         
        
    
    def getHitsPersecondGraphData(self):
        '''T=min,S=Seconds'''
        #self.df_jtl_HTTPReqeust['hits']=1
        self.df_hits=self.df_jtl_HTTPReqeust[['timeStamp','URL']]#self.df_jtl_HTTPReqeust[['timeStamp','hits']]
        #self.df_hits.sort_values(by=['timeStamp'])
        self.df_hits.set_index('timeStamp',inplace=True)
        self.df_hits_summary = pd.DataFrame()
        self.df_hits_summary['hitsCount']=self.df_hits['URL'].resample('1T').count()/60   
        #df.loc[(df!=0).any(axis=1)]
        self.df_hits_summary=self.df_hits_summary.loc[(self.df_hits_summary!=0).any(axis=1)]
        return self.df_hits_summary
        #return self.df_hits
    
    def getThrouhputPersecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_throughput=self.df_jtl_TC[['timeStamp','bytes']]
        #self.df_throughput.sort_values(by=['timeStamp'])
        self.df_throughput.set_index('timeStamp',inplace=True)
        self.df_throughput_summary = pd.DataFrame()
        self.df_throughput_summary['ThroughputReceived']=self.df_throughput['bytes'].resample('1T').sum()/60 
        return self.df_throughput_summary
        
    
    def getThrouhputSendPersecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_throughputsend=self.df_jtl_TC[['timeStamp','sentBytes']]
        #self.df_throughputsend.sort_values(by=['timeStamp'])
        self.df_throughputsend.set_index('timeStamp',inplace=True)
        self.df_throughputsend_summary = pd.DataFrame()
        self.df_throughputsend_summary['ThroughputSend']=self.df_throughputsend['sentBytes'].resample('1T').sum()/60    
        return self.df_throughputsend_summary
    
    def getOverallResponseTime(self):
        return self.df_jtl_TC[['timeStamp','elapsed','label']]
    
    def getRawDataFrame(self):
        return self.df_Jtl
    
    def temp(self):
        self.slicer_grp=self.df_jtl_TC[['timeStamp','elapsed','label']]
        self.slicer_grp.sort_values(by=['timeStamp'])
        self.slicer_grp.set_index('timeStamp',inplace=True)
        self.df_response_summary_slicer = pd.DataFrame()
        self.df_slicer_txnGroup=self.slicer_grp.groupby(['label'],observed=True )
        self.df_response_summary_slicer=self.df_slicer_txnGroup['elapsed'].resample('30T').agg(['min','mean','max','count'])
        return self.df_response_summary_slicer
    
    def getResponseMessage(self):
         return self.df_HttpErrors
    
    def junk(self):
        #return  self.df_jtl_TC
        return self.df_jtl_HTTPReqeust
        #return  self.df_Jtl
        #return self.df_hits
    
      
obj1=JTLReader("D:\\Projects\\JMR\\results\\14_Jan\\14_Jan\\90User_LoadTest_14Jan.jtl") 
#obj1.converTimeStamp()
obj1.ms_to_sec() 
obj1.getDuration()



'01:38:53'

In [117]:
#obj1.junk()

In [118]:
#y=obj1.getTotalPassFailFreqList()
#y

In [119]:
#x=obj1.temp()
#x

In [120]:
#obj1.df_Jtl

In [121]:
#obj1.df_jtl_HTTPReqeust

In [122]:
#obj1.df_jtl_TC

In [123]:
#obj1.getPeriod()

In [124]:
#obj1.getDuration()

In [125]:
#obj1.getMaxRunningVusers()

In [126]:
#obj1.getTotalThroughput()

In [127]:
"""Analysis Summery"""
analysisSummary=obj1.getAnalysisSummary()
analysisSummary


,Data,Values
0,Period,2021-01-14 13:43:10 : 2021-01-14 15:22:03
1,Duration,01:38:53
2,Maximum Running Vusers,35
3,Total bytes,476697584
4,Average bytes/second,80346.8
5,TotalHits,1400
6,Average Hits Per Second,0.235968
7,Total Error,NA
8,Total Pass,success False 657 True 650
9,Total Failed,NA


In [128]:
"""responseTimeSummary"""
responseTimeSummary=obj1.responseTimeSummary()
responseTimeSummary

,#Samples,Min,Average,90 Percentile,Max
label,,,,,
TC01_Txn01_URL,93.0,196.0,565.451613,1194.4,6200.0
TC01_Txn02_Login,93.0,204.0,6646.268817,19786.0,78039.0
TC01_Txn03_UploadFile,90.0,704.0,2041.644444,2558.1,21259.0
TC01_Txn04_03_Validate_5KRec,472.0,500.0,44980.338983,121156.2,125538.0
TC01_Txn04_02_Validate_1000Rec,295.0,183.0,56176.552542,120399.6,121469.0
TC01_Txn04_04_Validate_10KRec,168.0,862.0,101859.244048,122046.8,128945.0
TC01_Txn04_01_Validate_50Rec,96.0,159.0,20130.458333,33777.0,105763.0


In [129]:
"""Get Pass and Fail Summary"""
passandfail=obj1.getPassFailSummary()
passandfail

success,Fail,Pass
label,,
TC01_Txn01_URL,0.0,93.0
TC01_Txn02_Login,3.0,90.0
TC01_Txn03_UploadFile,3.0,87.0
TC01_Txn04_03_Validate_5KRec,319.0,153.0
TC01_Txn04_02_Validate_1000Rec,157.0,138.0
TC01_Txn04_04_Validate_10KRec,163.0,5.0
TC01_Txn04_01_Validate_50Rec,12.0,84.0


In [130]:
"""Get Final Response Summary"""
rs=obj1.getFinalResponseSummary()
rs

,#Samples,Pass,Fail,Error%,Min,Average,90 Percentile,Max
label,,,,,,,,
TC01_Txn01_URL,93.0,93.0,0.0,0.000000,196.0,565.451613,1194.4,6200.0
TC01_Txn02_Login,93.0,90.0,3.0,3.225806,204.0,6646.268817,19786.0,78039.0
TC01_Txn03_UploadFile,90.0,87.0,3.0,3.333333,704.0,2041.644444,2558.1,21259.0
TC01_Txn04_03_Validate_5KRec,472.0,153.0,319.0,67.584746,500.0,44980.338983,121156.2,125538.0
TC01_Txn04_02_Validate_1000Rec,295.0,138.0,157.0,53.220339,183.0,56176.552542,120399.6,121469.0
TC01_Txn04_04_Validate_10KRec,168.0,5.0,163.0,97.023810,862.0,101859.244048,122046.8,128945.0
TC01_Txn04_01_Validate_50Rec,96.0,84.0,12.0,12.500000,159.0,20130.458333,33777.0,105763.0


In [131]:
"""Get Vuser Summary"""
runningVuser=obj1.getRunnerVuserGraphData()
runningVuser

,Vusers
timeStamp,
2021-01-14 13:43:00,2.0
2021-01-14 13:44:00,14.0
2021-01-14 13:45:00,16.0
2021-01-14 13:46:00,18.0
2021-01-14 13:47:00,22.0
...,...
2021-01-14 15:18:00,8.0
2021-01-14 15:19:00,9.0
2021-01-14 15:20:00,9.0


In [132]:
"""Get Hits Summary"""
hitsPersecond=obj1.getHitsPersecondGraphData() 
hitsPersecond

,hitsCount
timeStamp,
2021-01-14 13:43:00,0.116667
2021-01-14 13:44:00,0.183333
2021-01-14 13:45:00,0.166667
2021-01-14 13:46:00,0.166667
2021-01-14 13:47:00,0.166667
...,...
2021-01-14 15:16:00,0.083333
2021-01-14 15:17:00,0.050000
2021-01-14 15:18:00,0.133333


In [133]:
"""Get Throughput Received Summery"""
throughput=obj1.getThrouhputPersecondGraphData()
throughput

,ThroughputReceived
timeStamp,
2021-01-14 13:43:00,89362.166667
2021-01-14 13:44:00,179340.133333
2021-01-14 13:45:00,178804.650000
2021-01-14 13:46:00,178805.033333
2021-01-14 13:47:00,178804.650000
...,...
2021-01-14 15:18:00,603.583333
2021-01-14 15:19:00,2106.366667
2021-01-14 15:20:00,0.000000


In [134]:
"""Get Throughput Send Summery"""
throughputsend=obj1.getThrouhputSendPersecondGraphData()
throughputsend

,ThroughputSend
timeStamp,
2021-01-14 13:43:00,281.133333
2021-01-14 13:44:00,44001.116667
2021-01-14 13:45:00,29492.383333
2021-01-14 13:46:00,29493.650000
2021-01-14 13:47:00,29493.500000
...,...
2021-01-14 15:18:00,145587.766667
2021-01-14 15:19:00,31189.350000
2021-01-14 15:20:00,0.000000


In [135]:
"""overAllResponseTimeSummary"""
overallResponseTime=obj1.getOverallResponseTime()
overallResponseTime

,timeStamp,elapsed,label
0,2021-01-14 13:43:10,750,TC01_Txn01_URL
3,2021-01-14 13:43:21,6324,TC01_Txn02_Login
5,2021-01-14 13:43:38,5091,TC01_Txn03_UploadFile
7,2021-01-14 13:43:40,518,TC01_Txn01_URL
10,2021-01-14 13:43:51,1673,TC01_Txn02_Login
...,...,...,...
2697,2021-01-14 15:19:02,27862,TC01_Txn04_01_Validate_50Rec
2700,2021-01-14 15:19:06,23583,TC01_Txn04_01_Validate_50Rec
2701,2021-01-14 15:19:06,23947,TC01_Txn04_01_Validate_50Rec
2703,2021-01-14 15:19:19,121098,TC01_Txn04_04_Validate_10KRec


In [136]:
rawdataframe=obj1.getRawDataFrame()
rawdataframe

,timeStamp,elapsed,label,responseCode,responseMessage,threadName,dataType,success,bytes,sentBytes,allThreads,URL
0,2021-01-14 13:43:10,0.750,TC01_Txn01_URL,200,"Number of samples in transaction : 2, number o...",5000Record 1-1,NaN,True,19731,1330,1,NaN
1,2021-01-14 13:43:10,0.693,TC01_Txn01_URL-0,200,OK,5000Record 1-1,text,True,10530,721,1,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
2,2021-01-14 13:43:11,0.057,TC01_Txn01_URL-1,200,OK,5000Record 1-1,text,True,9201,609,1,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
3,2021-01-14 13:43:21,6.324,TC01_Txn02_Login,200,"Number of samples in transaction : 1, number o...",5000Record 1-1,NaN,True,10150,1359,1,NaN
4,2021-01-14 13:43:21,6.324,TC01_Txn02_Login-0,200,OK,5000Record 1-1,text,True,10150,1359,1,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2021-01-14 15:19:06,23.947,TC01_Txn04_01_Validate_50Rec-0,200,OK,50Record 1-3,text,True,31403,36514,9,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
2703,2021-01-14 15:19:19,121.098,TC01_Txn04_04_Validate_10KRec,503,NaN,10000Record 1-15,NaN,False,684,1725106,1,NaN
2704,2021-01-14 15:19:19,121.097,TC01_Txn04_04_Validate_10KRec-0,503,Service Unavailable,10000Record 1-15,text,False,684,1725106,1,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
2705,2021-01-14 15:22:03,13.282,TC01_Txn04_01_Validate_50Rec,200,NaN,50Record 1-8,NaN,True,31404,36515,4,NaN


In [137]:
"""Generate Excel Report"""
writer = ExcelWriter('JTL_Log_Analyser_Output.xlsx',mode='w')
#Write Analysis Summary to Excel
analysisSummary.to_excel(writer,sheet_name='AnalysisSummary')
#write response summary to excel with txn name,min,avg,max,90% & max
rs.to_excel(writer,sheet_name='ResponseSummary')
#Write Running Vusers
runningVuser.to_excel(writer,sheet_name='RunningVuserSummary')
#Write Hits Summary
hitsPersecond.to_excel(writer,sheet_name='HitsSummary')
#Write Throughput Received Summary
throughput.to_excel(writer,sheet_name='ThroughputReceivedSummary')
#Write Throughput Send Summary
throughputsend.to_excel(writer,sheet_name='ThroughputSendSummary')
#Write ResponseTime
overallResponseTime.to_excel(writer,sheet_name='ResponseTimeGraph')
#Export Original dataframe
rawdataframe.to_excel(writer,sheet_name='dataframe')
writer.save()







# obj1=JTLReader("D:\\Projects\\JMR\\results\\50UserLoadtest_jtl\\50UserLoadtest_jtl\\LT_50Users.jtl") 
obj1.ms_to_sec() 
obj1.getDuration()
analysisSummary=obj1.getAnalysisSummary()
responseTimeSummary=obj1.responseTimeSummary()
passandfail=obj1.getPassFailSummary()
rs=obj1.getFinalResponseSummary()
runningVuser=obj1.getRunnerVuserGraphData()
hitsPersecond=obj1.getHitsPersecondGraphData() 
throughput=obj1.getThrouhputPersecondGraphData()
overallResponseTime=obj1.getOverallResponseTime()
writer = ExcelWriter('JTL_Log_Analyser_Output.xlsx',mode='w')
analysisSummary.to_excel(writer,sheet_name='AnalysisSummary')
rs.to_excel(writer,sheet_name='ResponseSummary')
runningVuser.to_excel(writer,sheet_name='RunningVuserSummary')
hitsPersecond.to_excel(writer,sheet_name='HitsSummary')
throughput.to_excel(writer,sheet_name='ThroughputSummary')
overallResponseTime.to_excel(writer,sheet_name='ResponseTimeGraph')
writer.save()

In [93]:
responseMessage=obj1.getResponseMessage()
responseMessage
#responseMessage_grp=responseMessage.groupby('label',observed=True )
#errorSummary=responseMessage_grp['responseMessage'].value_counts()
#errorSummary.unstack()
#responseMessage=responseMessage.set_option('display.max_colwidth',1000)
#x=responseMessage[['label','responseMessage']]
#x.loc[227077,'responseMessage']

#responseMessage['responseMessage']
#'Non HTTP response message: Address already in use: connect'





,timeStamp,elapsed,label,responseCode,responseMessage,threadName,dataType,success,bytes,sentBytes,allThreads,URL
198,2021-01-14 13:53:21,702,TC01_Txn02_Login-0,200,OK,5000Record 1-21,text,False,3615,1357,21,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
475,2021-01-14 14:09:38,3375,TC01_Txn04_03_Validate_5KRec-0,Non HTTP response code: java.net.SocketException,Non HTTP response message: Connection reset by...,5000Record 1-28,text,False,3406,0,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
476,2021-01-14 14:09:38,3369,TC01_Txn04_03_Validate_5KRec-0,Non HTTP response code: java.net.SocketException,Non HTTP response message: Connection reset by...,5000Record 1-6,text,False,3406,0,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
480,2021-01-14 14:09:38,3341,TC01_Txn04_03_Validate_5KRec-0,Non HTTP response code: java.net.SocketException,Non HTTP response message: Connection reset by...,5000Record 1-16,text,False,3406,0,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
481,2021-01-14 14:09:38,3342,TC01_Txn04_03_Validate_5KRec-0,Non HTTP response code: java.net.SocketException,Non HTTP response message: Connection reset by...,5000Record 1-27,text,False,3406,0,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
...,...,...,...,...,...,...,...,...,...,...,...,...
1185,2021-01-14 15:08:05,120693,TC01_Txn04_03_Validate_5KRec-0,503,Service Unavailable,5000Record 1-3,text,False,684,870588,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
1187,2021-01-14 15:08:08,120635,TC01_Txn04_03_Validate_5KRec-0,503,Service Unavailable,5000Record 1-11,text,False,684,870590,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
1189,2021-01-14 15:09:20,675,TC01_Txn04_03_Validate_5KRec-0,200,OK,5000Record 1-33,text,False,675,870656,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...
1190,2021-01-14 15:09:28,628,TC01_Txn04_03_Validate_5KRec-0,200,OK,5000Record 1-23,text,False,675,870631,35,https://audipay.bankaudi.com.eg/AUDIPAY/faces/...


In [ ]:
import random
import matplotlib.pyplot as plt
#from matplotlib import style
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
%matplotlib inline


#print(plt.style.available)
#print(plt.__file__)

#style.use('dark_background')
plt.rcParams['figure.figsize']=[25,18]
fig = plt.figure(figsize=(20,10))
fig.set_facecolor('#003366')

# Add suplot syntax

ax1 = fig.add_subplot(211)

#To get data for x and y axis
x,y=Throughput.index,Throughput['Throughput']
#To rotate 
for label in ax1.xaxis.get_ticklabels():
    label.set_rotation(0)

#To print x axis with date and time
xfmt = mdates.DateFormatter('%H:%M:%S')#%d-%m-%y %H:%M:%S//ax.xaxis_date()
ax1.xaxis.set_major_formatter(xfmt)
ax1.fill_between(x,y,color="red",linewidth=5)#https://matplotlib.org/users/dflt_style_changes.html

# x and y label
ax1.set_xlabel('Time',fontsize=20,color='white')#alpha=0.3
ax1.set_ylabel('Throughput/sec',fontsize=20)
ax1.set_title('Throughput',fontsize=20,color='white')

#To drwa 4 lines with the desier colour : Top,Bottom,Left and Right
ax1.spines['bottom'].set_color('black')
ax1.spines['top'].set_color('black')
ax1.spines['left'].set_color('black')
ax1.spines['right'].set_color('black')

#
ax1.xaxis.label.set_color('white')#"Time"
ax1.yaxis.label.set_color('white')#"HitsPerMin"

ax1.tick_params(axis='x', labelcolor='white',labelsize=20)#"%d-%m-%y %H:%M:%S"
ax1.tick_params(axis='y', labelcolor='white',labelsize=20)#"Hits Count: 10,20,30,40,50,60,70"

ax1.set_facecolor('#003366')#https://matplotlib.org/users/colors.html

##003366

ax1.grid(which='major', linestyle='-', linewidth='0.2', color='white')#17becf
#ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='red')

#ax1.set_yticks([0,1,2,3,4,5,6,7,8,9,10])
#ax1.set_xticks(['22:00:00','22:30:00','23:00:00','23:30:00','00:00:00',
               # '00:30:00','01:00:00','01:30:00','02:00:00','02:30:00',
               # '03:00:00','03:30:00','04:00:00','04:30:00'])
#ax.set_ylim((0, 6))
#ax.set_xlim()

#fig.savefig("E:\MaxUser.pdf",dpi=None,facecolor='#003366', edgecolor='#003366',bbox_inches='tight')
plt.show()